In [2]:
# Install necessary packages
!pip install datasets transformers openai-whisper jiwer
!pip install soundfile

from datasets import load_dataset, load_metric
import numpy as np
import whisper
import jiwer

# Load the LibriSpeech dataset (using a small subset for demonstration)
dataset = load_dataset("librispeech_asr", "clean", split="test[:1%]", trust_remote_code=True)

# Load the pre-trained Whisper model
model = whisper.load_model("base")

# Function to transcribe audio using the Whisper model
def transcribe(batch):
    audio = batch["audio"]
    # Whisper model expects 16kHz audio
    result = model.transcribe(audio["array"], fp16=False)
    batch["transcription"] = result["text"]
    return batch

# Apply transcription to the dataset
dataset = dataset.map(transcribe)

# Load the CER metric
cer_metric = load_metric("cer")

# Compute CER
transcriptions = dataset["transcription"]
references = dataset["text"]
cer = cer_metric.compute(predictions=transcriptions, references=references)

print(f"CER: {cer:.4f}")


Generating train.100 split:   0%|          | 0/28539 [00:00<?, ? examples/s]


DatasetGenerationError: An error occurred while generating the dataset